In [1]:
import pandas as pd
from sklearn.utils import shuffle

In [2]:
data_rt = pd.read_csv("C:/Users/Desktop/UDSC/Sentiment analysis/Data/reviews_rt_all.csv", sep="|")
data_imdb = pd.read_csv("C:/Users/Desktop/UDSC/Sentiment analysis/Data/imdb_small.csv", sep="|")

data_df = pd.concat([data_rt, data_imdb], ignore_index=True, copy=False)
data_df = shuffle(data_df)

In [3]:
import re, nltk
from nltk.stem import SnowballStemmer

In [4]:
def tokenize(text):
    text = re.sub("[^a-zA-Z]", " ", text)
    word_list = nltk.word_tokenize(text)
    stemmer = SnowballStemmer("english")
    stems = [stemmer.stem(word) for word in word_list]
    return stems

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(tokenizer=tokenize, ngram_range=(1, 3),
                              analyzer = 'word', binary = True, max_df = 0.75)

data_features = vectorizer.fit_transform(data_df['text'])

In [6]:
from sklearn.model_selection import train_test_split

X_train_rt, X_test_rt, y_train_rt, y_test_rt  = train_test_split(
        data_rt.text, 
        data_rt.label,
        test_size=0.2, 
        random_state=42)

In [7]:
X_train_imdb, X_test_imdb, y_train_imdb, y_test_imdb  = train_test_split(
        data_imdb.text, 
        data_imdb.label,
        test_size=0.2, 
        random_state=42)

In [8]:
X_train = pd.concat([X_train_rt, X_train_imdb])
X_test = pd.concat([X_test_rt, X_test_imdb])
y_train = pd.concat([y_train_rt, y_train_imdb])
y_test = pd.concat([y_test_rt, y_test_imdb])

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('vectorizer', TfidfVectorizer(tokenizer=tokenize, ngram_range=(1, 3),
                              analyzer = 'word', binary = True, max_df = 0.75)), 
                     ('classifier', LogisticRegression(C = 100))])
model = pipeline.fit(X=X_train, y=y_train)

In [10]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)

print (accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.848895878383
             precision    recall  f1-score   support

          0       0.84      0.79      0.81     12576
          1       0.86      0.89      0.87     17946

avg / total       0.85      0.85      0.85     30522



In [12]:
y_pred = model.predict(X_test_rt)

print (accuracy_score(y_test_rt, y_pred))
print(classification_report(y_test_rt, y_pred))

0.815953610759
             precision    recall  f1-score   support

          0       0.77      0.70      0.74      7521
          1       0.84      0.88      0.86     13001

avg / total       0.81      0.82      0.81     20522



In [13]:
y_pred = model.predict(X_test_imdb)

print (accuracy_score(y_test_imdb, y_pred))
print(classification_report(y_test_imdb, y_pred))

0.9165
             precision    recall  f1-score   support

          0       0.92      0.92      0.92      5055
          1       0.91      0.92      0.92      4945

avg / total       0.92      0.92      0.92     10000

